In [1]:
from google.colab import drive
drive.mount('/content/drive')

root = "drive/My Drive/"
image_dir = "MICCAI_BraTS_2018_Data_Training/"
hgg = "HGG/"
lgg = "LGG/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:

import pickle

from functools import partial

from keras.layers import Input, LeakyReLU, Add, UpSampling3D, Activation, SpatialDropout3D
from keras.engine import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop

import pandas as pd

import numpy as np
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import *

# K.set_image_dim_ordering('th')
# K.set_image_dim_ordering('tf')
# K.set_image_data_format('channels_first')

try:
    from keras.engine import merge
except ImportError:
    from keras.layers.merge import concatenate

from tensorflow.python.client import device_lib
import numpy as np
!pip install SimpleITK
!pip install nilearn
import SimpleITK as sitk
import skimage.io as io
from nilearn.image.image import check_niimg
from nilearn.image.image import _crop_img_to as crop_img_to

import nibabel as nib
import tensorflow as tf
import skimage.io as io
import os

     |████████████████████████████████| 42.5MB 99kB/s 
     |████████████████████████████████| 2.5MB 9.1MB/s 


In [3]:
def dice_coefficient(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coefficient_loss(y_true, y_pred):
    return -dice_coefficient(y_true, y_pred)


def weighted_dice_coefficient(y_true, y_pred, axis=(-3, -2, -1), smooth=0.00001):

    return K.mean(2. * (K.sum(y_true * y_pred,
                              axis=axis) + smooth/2)/(K.sum(y_true,
                                                            axis=axis) + K.sum(y_pred,
                                                                               axis=axis) + smooth))
def weighted_dice_coefficient_loss(y_true, y_pred):
    return -weighted_dice_coefficient(y_true, y_pred)


def label_wise_dice_coefficient(y_true, y_pred, label_index):
    return dice_coefficient(y_true[:, label_index], y_pred[:, label_index])


def get_label_dice_coefficient_function(label_index):
    f = partial(label_wise_dice_coefficient, label_index=label_index)
    f.__setattr__('__name__', 'label_{0}_dice_coef'.format(label_index))
    return f


In [4]:
from nilearn.image.image import check_niimg
from nilearn.image.image import _crop_img_to as crop_img_to


BLOCKSIZE = (32, 160, 160)

def crop_img(img, rtol=1e-8, copy=True, return_slices=False):
  img = check_niimg(img)
  data = img.get_data()
  infinity_norm = max(-data.min(), data.max())
  passes_threshold = np.logical_or(data < -rtol * infinity_norm,
                                    data > rtol * infinity_norm)

  if data.ndim == 4:
      passes_threshold = np.any(passes_threshold, axis=-1)
  coords = np.array(np.where(passes_threshold))
  start = coords.min(axis=1)
  end = coords.max(axis=1) + 1

  # pad with one voxel to avoid resampling problems
  start = np.maximum(start - 1, 0)
  end = np.minimum(end + 1, data.shape[:3])

  slices = [slice(s, e) for s, e in zip(start, end)]

  if return_slices:
      return slices

  return crop_img_to(img, slices, copy=copy)

def normalize(slice, bottom=99, down=1):

    b = np.percentile(slice, bottom)
    t = np.percentile(slice, down)
    slice = np.clip(slice, t, b)

    image_nonzero = slice[np.nonzero(slice)]
    if np.std(slice) == 0 or np.std(image_nonzero) == 0:
        return slice
    else:
        tmp = (slice - np.mean(image_nonzero)) / np.std(image_nonzero)
        tmp[tmp == tmp.min()] = -9
        return tmp

def insert_slices(array):
  myblackslice = np.zeros([240,240])
  array = np.insert(array,0,myblackslice,axis = 0)
  array = np.insert(array,0,myblackslice,axis = 0)
  array = np.insert(array,0,myblackslice,axis = 0)
  array = np.insert(array,array.shape[0],myblackslice,axis = 0)
  array = np.insert(array,array.shape[0],myblackslice,axis = 0)

  array_nor = normalize(array)
  crop = crop_ceter(array_nor,160,160)

  patch_block_size = BLOCKSIZE
  numberxy = patch_block_size[1]
  numberz = patch_block_size[0]
  width = np.shape(crop)[1]
  height = np.shape(crop)[2]
  imagez = np.shape(crop)[0]
  block_width = np.array(patch_block_size)[1]
  block_height = np.array(patch_block_size)[2]
  blockz = np.array(patch_block_size)[0]
  stridewidth = (width - block_width) // numberxy
  strideheight = (height - block_height) // numberxy
  stridez = (imagez - blockz) // numberz
  step_width = width - (stridewidth * numberxy + block_width)
  step_width = step_width // 2
  step_height = height - (strideheight * numberxy + block_height)
  step_height = step_height // 2
  step_z = imagez - (stridez * numberz + blockz)
  step_z = step_z // 2

  hr_samples_list = []
  patchnum = []
  for z in range(step_z, numberz * (stridez + 1) + step_z, numberz):
    for x in range(step_width, numberxy * (stridewidth + 1) + step_width, numberxy):
      for y in range(step_height, numberxy * (strideheight + 1) + step_height, numberxy):
        patchnum.append(z)
        hr_samples_list.append(crop[z:z + blockz, x:x + block_width, y:y + block_height])
  samples_arr = np.array(hr_samples_list).reshape((len(hr_samples_list), blockz, block_width, block_height))

  return samples_arr

def merge(samples, imagez, height, width, samples_flair, samples_t1, samples_t1ce, samples_t2, mask_samples, ID):
  for j in range(samples):

    fourmodelimagearray = np.zeros((imagez, height, width, 4), np.float)
    flairimage = samples_flair[j, :, :, :]
    flairimage = flairimage.astype(np.float)
    fourmodelimagearray[:, :, :, 0] = flairimage
    t1image = samples_t1[j, :, :, :]
    t1image = t1image.astype(np.float)
    fourmodelimagearray[:, :, :, 1] = t1image
    t1ceimage = samples_t1ce[j, :, :, :]
    t1ceimage = t1ceimage.astype(np.float)
    fourmodelimagearray[:, :, :, 2] = t1ceimage
    t2image = samples_t2[j, :, :, :]
    t2image = t2image.astype(np.float)
    fourmodelimagearray[:, :, :, 3] = t2image
    
    wt_tc_etMaskArray = np.zeros((imagez, height, width, 3), np.uint8)
    mask_one_sample = mask_samples[j, :, :, :]
    WT_Label = mask_one_sample.copy()
    WT_Label[mask_one_sample == 1] = 1.
    WT_Label[mask_one_sample == 2] = 1.
    WT_Label[mask_one_sample == 4] = 1.
    TC_Label = mask_one_sample.copy()
    TC_Label[mask_one_sample == 1] = 1.
    TC_Label[mask_one_sample == 2] = 0.
    TC_Label[mask_one_sample == 4] = 1.
    ET_Label = mask_one_sample.copy()
    ET_Label[mask_one_sample == 1] = 0.
    ET_Label[mask_one_sample == 2] = 0.
    ET_Label[mask_one_sample == 4] = 1.
    wt_tc_etMaskArray[:, :, :, 0] = WT_Label
    wt_tc_etMaskArray[:, :, :, 1] = TC_Label
    wt_tc_etMaskArray[:, :, :, 2] = ET_Label

    # fourmodelimagearray = np.rollaxis(fourmodelimagearray, 1, 0)
    # wt_tc_etMaskArray = np.rollaxis(wt_tc_etMaskArray, 1, 0)

    pickle.dump( fourmodelimagearray, open( f"{ID}{j}_images.pkl", "wb" ) )
    pickle.dump( wt_tc_etMaskArray, open( f"{ID}{j}_seg_mask_3ch.pkl", "wb" ) )
    name = ID + str(j)
    print(fourmodelimagearray.shape)
    print(wt_tc_etMaskArray.shape)
    completelist.append(name)
    # return fourmodelimagearray, wt_tc_etMaskArray

def crop_ceter(img,croph,cropw):   
    #for n_slice in range(img.shape[0]):
    height,width = img[0].shape 
    starth = height//2-(croph//2)
    startw = width//2-(cropw//2)        
    return img[:,starth:starth+croph,startw:startw+cropw]

In [5]:
files1 = os.listdir(root + image_dir + hgg)
files2 = os.listdir(root + image_dir + lgg)

def read_image(path):
  img = sitk.ReadImage(path)
  data = sitk.GetArrayFromImage(img)
  return data

HGG_dir_list = []
LGG_dir_list = []
completelist = []

for file in files1[1:]:
  file = file + '/'
  sub_files = os.listdir(root + image_dir + hgg + file)
  HGG_dir_list.append(file)
  flair = []
  seg = []
  t1 = []
  t1ce = []
  t2 = []

  for fil in sub_files:

    if 'flair.nii' in fil:
      data = read_image(root + image_dir + hgg + file + fil)
      img = insert_slices(data)
      # img = normalize(img)
      flair.append(img)
    elif 'seg.nii' in fil:
      data = read_image(root + image_dir + hgg + file + fil)
      img = insert_slices(data)
      # img = normalize(img)
      seg.append(img)
    elif 't1.nii' in fil:
      data = read_image(root + image_dir + hgg + file + fil)
      img = insert_slices(data)
      # img = normalize(img)
      t1.append(img)
    elif 't1ce.nii' in fil:
      data = read_image(root + image_dir + hgg + file + fil)
      img = insert_slices(data)
      # img = normalize(img)
      t1ce.append(img)
    elif 't2.nii' in fil:
      data = read_image(root + image_dir + hgg + file + fil)
      img = insert_slices(data)
      # img = normalize(img)
      t2.append(img)

  ID = file.replace('/', '')

  samples, imagez, height, width = np.shape(flair[0])[0], np.shape(flair[0])[1], np.shape(flair[0])[2], np.shape(flair[0])[3]
  print(samples)
  merge(samples, imagez, height, width, flair[0], t1[0], t1ce[0], t2[0], seg[0], ID)  

for file in files2[1:]:
  file = file + '/'
  sub_files = os.listdir(root + image_dir + lgg + file)
  LGG_dir_list.append(file)

  flair = []
  seg = []
  t1 = []
  t1ce = []
  t2 = []

  for fil in sub_files:

    if 'flair.nii' in fil:
      data = read_image(root + image_dir + lgg + file + fil)
      img = insert_slices(data)
      # img = normalize(img)
      flair.append(img)
    elif 'seg.nii' in fil:
      data = read_image(root + image_dir + lgg + file + fil)
      img = insert_slices(data)
      # img = normalize(img)
      seg.append(img)
    elif 't1.nii' in fil:
      data = read_image(root + image_dir + lgg + file + fil)
      img = insert_slices(data)
      # img = normalize(img)
      t1.append(img)
    elif 't1ce.nii' in fil:
      data = read_image(root + image_dir + lgg + file + fil)
      img = insert_slices(data)
      # img = normalize(img)
      t1ce.append(img)
    elif 't2.nii' in fil:
      data = read_image(root + image_dir + lgg + file + fil)
      img = insert_slices(data)
      # img = normalize(img)
      t2.append(img)

  ID = file.replace('/', '')
  samples, imagez, height, width = np.shape(flair[0])[0], np.shape(flair[0])[1], np.shape(flair[0])[2], np.shape(flair[0])[3]

  merge(samples, imagez, height, width, flair[0], t1[0], t1ce[0], t2[0], seg[0], ID)

  # # fourModels = np.rollaxis(fourModels, 2, 0)
  # fourModels = np.rollaxis(fourModels, 1, 0)
  # # masks = np.rollaxis(masks, 2, 0)
  # masks = np.rollaxis(masks, 1, 0)

  # pickle.dump( fourModels, open( f"{ID}_images.pkl", "wb" ) )
  # pickle.dump( masks, open( f"{ID}_seg_mask_3ch.pkl", "wb" ) ) 



5
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
5
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
5
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
5
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
5
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
5
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4)
(32, 160, 160, 3)
(32, 160, 160, 4

In [6]:
from pathlib import Path

tumor_type_dict = {}

for patientID in HGG_dir_list:
  patientID = patientID.replace('/','')
  tumor_type_dict[patientID] = 0
for patientID in LGG_dir_list:
  patientID = patientID.replace('/','')
  tumor_type_dict[patientID] = 1

# completelist = HGG_dir_list + LGG_dir_list
np.random.shuffle(completelist)

# for patientID in completelist:
#   print(tumor_type_dict[patientID])
train_val_test_dict={}

holdout_percentage=0.15
train_val_test_dict['test']=completelist[0:int(len(completelist)*holdout_percentage)]
trainlist=completelist[int(len(completelist)*holdout_percentage):len(completelist)]

train_percentage=0.7
train_val_test_dict['train']=trainlist[0:int(len(trainlist)*train_percentage)]
train_val_test_dict['val']=trainlist[int(len(trainlist)*train_percentage):len(trainlist)]

pickle.dump( tumor_type_dict, open( "tumor_type_dict.pkl", "wb" ) )
pickle.dump( train_val_test_dict, open( "train_val_test_dict.pkl", "wb" ) )

In [7]:
from keras.layers import Layer, InputSpec
from keras import initializers, regularizers, constraints
from keras import backend as K


class InstanceNormalization(Layer):
            
    def __init__(self,
                 axis=None,
                 epsilon=1e-3,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = initializers.get(beta_initializer)
        self.gamma_initializer = initializers.get(gamma_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)

    def build(self, input_shape):
        ndim = len(input_shape)
        if self.axis == 0:
            raise ValueError('Axis cannot be zero')

        if (self.axis is not None) and (ndim == 2):
            raise ValueError('Cannot specify axis for rank 1 tensor')

        self.input_spec = InputSpec(ndim=ndim)

        if self.axis is None:
            shape = (1,)
        else:
            shape = (input_shape[self.axis],)

        if self.scale:
            self.gamma = self.add_weight(shape=shape,
                                         name='gamma',
                                         initializer=self.gamma_initializer,
                                         regularizer=self.gamma_regularizer,
                                         constraint=self.gamma_constraint)
        else:
            self.gamma = None
        if self.center:
            self.beta = self.add_weight(shape=shape,
                                        name='beta',
                                        initializer=self.beta_initializer,
                                        regularizer=self.beta_regularizer,
                                        constraint=self.beta_constraint)
        else:
            self.beta = None
        self.built = True

    def call(self, inputs, training=None):
        input_shape = K.int_shape(inputs)
        reduction_axes = list(range(0, len(input_shape)))

        if self.axis is not None:
            del reduction_axes[self.axis]

        del reduction_axes[0]

        mean = K.mean(inputs, reduction_axes, keepdims=True)
        stddev = K.std(inputs, reduction_axes, keepdims=True) + self.epsilon
        normed = (inputs - mean) / stddev

        broadcast_shape = [1] * len(input_shape)
        if self.axis is not None:
            broadcast_shape[self.axis] = input_shape[self.axis]

        if self.scale:
            broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
            normed = normed * broadcast_gamma
        if self.center:
            broadcast_beta = K.reshape(self.beta, broadcast_shape)
            normed = normed + broadcast_beta
        return normed

    def get_config(self):
        config = {
            'axis': self.axis,
            'epsilon': self.epsilon,
            'center': self.center,
            'scale': self.scale,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'beta_constraint': constraints.serialize(self.beta_constraint),
            'gamma_constraint': constraints.serialize(self.gamma_constraint)
        }
        base_config = super(InstanceNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [8]:
def create_convolution_block(input_layer, n_filters, batch_normalization=False, kernel=(3, 3, 3), activation=None,
                             padding='same', strides=(1, 1, 1), instance_normalization=False):

    layer = Conv3D(n_filters, kernel, padding=padding, strides=strides)(input_layer)
    if batch_normalization:
        layer = BatchNormalization(axis=1)(layer)
    # elif instance_normalization:
    #     # try:
    #     #     from keras_contrib.layers.normalization import InstanceNormalization
    #     # except ImportError:
    #     #     raise ImportError("Install keras_contrib in order to use instance normalization."
    #     #                       "\nTry: pip install git+https://www.github.com/farizrahman4u/keras-contrib.git")
    #     layer = InstanceNormalization(axis=1)(layer)
    if activation is None:
        return Activation('relu')(layer)
    else:
        return activation()(layer)

def compute_level_output_shape(n_filters, depth, pool_size, image_shape):

    output_image_shape = np.asarray(np.divide(image_shape, np.power(pool_size, depth)), dtype=np.int32).tolist()
    return tuple([None, n_filters] + output_image_shape)


def get_up_convolution(n_filters, pool_size, kernel_size=(2, 2, 2), strides=(2, 2, 2),
                       deconvolution=False):
    if deconvolution:
        return Deconvolution3D(filters=n_filters, kernel_size=kernel_size,
                               strides=strides)
    else:
        return UpSampling3D(size=pool_size)

def create_localization_module(input_layer, n_filters):
    convolution1 = create_convolution_block(input_layer, n_filters)
    convolution2 = create_convolution_block(convolution1, n_filters, kernel=(1, 1, 1))
    return convolution2


def create_up_sampling_module(input_layer, n_filters, size=(2, 2, 2)):
    up_sample = UpSampling3D(size=size)(input_layer)
    convolution = create_convolution_block(up_sample, n_filters)
    return convolution

def create_context_module(input_layer, n_level_filters, dropout_rate=0.3, data_format="channels_first"):
    convolution1 = create_convolution_block(input_layer=input_layer, n_filters=n_level_filters)
    dropout = SpatialDropout3D(rate=dropout_rate, data_format=data_format)(convolution1)
    convolution2 = create_convolution_block(input_layer=dropout, n_filters=n_level_filters)
    return convolution2


create_convolution_block = partial(create_convolution_block, activation=LeakyReLU, instance_normalization=True)

In [9]:
import numpy as np
import keras
import nibabel as nib

class SingleDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=2, dim=(240,240,155), n_channels=4,
                 n_classes=3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        # self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y1 = self.__data_generation(list_IDs_temp)

        return X, y1

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y1 = np.empty((self.batch_size, *self.dim, 3))

        # Generate data
        # Decode and load the data
        for i, ID in enumerate(list_IDs_temp):
            
            ID = ID.replace('/','')
            
            X[i,] = pickle.load( open("%s_images.pkl"%(ID), "rb"))
            y1[i,] = pickle.load( open("%s_seg_mask_3ch.pkl"%(ID), "rb"))
            
        return X, y1

In [10]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

input_shape=(32, 160, 160, 4)
n_base_filters=12
depth=5
dropout_rate=0.3
n_segmentation_levels=3
n_labels=3
activation_name="sigmoid"

inputs = Input(input_shape)

current_layer = inputs
level_output_layers = list()
level_filters = list()
for level_number in range(depth):
    n_level_filters = (2**level_number) * n_base_filters
    level_filters.append(n_level_filters)

    if current_layer is inputs:
        in_conv = create_convolution_block(current_layer, n_level_filters)
    else:
        in_conv = create_convolution_block(current_layer, n_level_filters, strides=(2, 2, 2))

    context_output_layer = create_context_module(in_conv, n_level_filters, dropout_rate=dropout_rate)
    summation_layer = Add()([in_conv, context_output_layer])
    level_output_layers.append(summation_layer)
    current_layer = summation_layer

segmentation_layers = list()
for level_number in range(depth - 2, -1, -1):
    up_sampling = create_up_sampling_module(current_layer, level_filters[level_number])
    concatenation_layer = concatenate([level_output_layers[level_number], up_sampling], axis=4)
    localization_output = create_localization_module(concatenation_layer, level_filters[level_number])
    current_layer = localization_output
    if level_number < n_segmentation_levels:
        segmentation_layers.insert(0, create_convolution_block(current_layer, n_filters=n_labels, kernel=(1, 1, 1)))

output_layer = None
for level_number in reversed(range(n_segmentation_levels)):
    segmentation_layer = segmentation_layers[level_number]
    if output_layer is None:
        output_layer = segmentation_layer
    else:
        output_layer = Add()([output_layer, segmentation_layer])

    if level_number > 0:      
        output_layer = UpSampling3D(size=(2, 2, 2))(output_layer)

activation_block = Activation(activation = activation_name, name='activation_block')(output_layer)

survival_conv_1 = Conv3D(filters=n_level_filters, kernel_size=(3, 3, 3), padding='same', strides=(1, 1, 1), name='survival_conv_1')(summation_layer)
survival_conv_2 = Conv3D(filters=n_level_filters, kernel_size=(3, 3, 3), padding='same', strides=(1, 1, 1), name='survival_conv_2')(survival_conv_1)
dropout = SpatialDropout3D(rate=dropout_rate, data_format='channels_first', name='dropout')(survival_conv_2)
survival_conv_3 = Conv3D(filters=n_level_filters, kernel_size=(3, 3, 3), padding='same', strides=(1, 1, 1), name='survival_conv_3')(dropout)
survival_GAP = GlobalAveragePooling3D(name='survival_GAP')(survival_conv_3)
survival_block = Dense(1, activation='linear', name='survival_block')(survival_GAP)

tumortype_conv_1 = Conv3D(filters=n_level_filters, kernel_size=(3, 3, 3), padding='same', strides=(1, 1, 1), name='tumortype_conv_1')(summation_layer)
tumortype_conv_2 = Conv3D(filters=n_level_filters, kernel_size=(3, 3, 3), padding='same', strides=(1, 1, 1), name='tumortype_conv_2')(tumortype_conv_1)
tumortype_dropout = SpatialDropout3D(rate=dropout_rate, data_format='channels_first', name='tumortype_dropout')(tumortype_conv_2)
tumortype_conv_3 = Conv3D(filters=n_level_filters, kernel_size=(3, 3, 3), padding='same', strides=(1, 1, 1), name='tumortype_conv_3')(tumortype_dropout)
tumortype_GAP = GlobalAveragePooling3D(name='tumortype_GAP')(tumortype_conv_3)
tumortype_block = Dense(1, activation='sigmoid', name='tumortype_block')(tumortype_GAP)  

model = Model(inputs=inputs, outputs=[activation_block])

model.compile(optimizer=Adam(lr=1e-2), 
              loss={'activation_block': weighted_dice_coefficient_loss}, 
              loss_weights={'activation_block': 1.},
             metrics={'activation_block': ['accuracy',weighted_dice_coefficient, dice_coefficient]})

model.summary(line_length=150)

Instructions for updating:
non-resource variables are not supported in the long term
Model: "functional_1"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_1 (InputLayer)                             [(None, 32, 160, 160, 4)]        0                                                                   
______________________________________________________________________________________________________________________________________________________
conv3d (Conv3D)                                  (None, 32, 160, 160, 12)         1308              input_1[0][0]                                     
__________________________________________________________________________________________________________________________________________

In [ ]:

params = {'dim': (32,160,160),
          'batch_size': 1,
          'n_classes': 3,
          'n_channels': 4,
          'shuffle': True}

# Generators
print(len(train_val_test_dict['train']))
training_generator = SingleDataGenerator(train_val_test_dict['train'], **params)
validation_generator = SingleDataGenerator(train_val_test_dict['val'], **params)

cb_1=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
cb_2=keras.callbacks.ModelCheckpoint(filepath="1Bpred_weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)


results = model.fit_generator(generator=training_generator,
                        validation_data=validation_generator,
                    epochs=100,
                    callbacks=[cb_1,cb_2])
model.save_weights("model_1_weights.h5")

848
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
160/848 [====>.........................] - ETA: 3:52:58 - batch: 79.5000 - size: 1.0000 - loss: -0.3672 - acc: 0.9998 - weighted_dice_coefficient: 0.3672 - dice_coefficient: 0.3673

In [ ]:
history_1_pred = results.history
pickle.dump( history_1_pred, open( "history_1_pred.pkl", "wb" ) )

params = {'dim': (160,32,160),
          'batch_size': 1,
          'n_classes': 3,
          'n_channels': 4,
          'shuffle': False}


validation_generator = SingleDataGenerator(train_val_test_dict['test'], **params)

predictions_1_pred = model.predict_generator(generator=validation_generator)

pickle.dump( predictions_1_pred, open( "predictions_1_pred.pkl", "wb" ) )